In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!pip uninstall keras tensorflow -y
!pip install keras==2.2.5 tensorflow==1.13.1
!pip install pydload
!pip install flask_cors
!pip install flask-ngrok

Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1
Uninstalling tensorflow-2.2.0:
  Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 337kB 2.7MB/s 
     |████████████████████████████████| 92.5MB 108kB/s 
     |████████████████████████████████| 368kB 45.2MB/s 
     |████████████████████████████████| 3.2MB 45.4MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1


In [2]:
!pip freeze

absl-py==0.9.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
asgiref==3.2.7
astor==0.8.1
astropy==4.0.1.post1
astunparse==1.6.3
atari-py==0.2.6
atomicwrites==1.4.0
attrs==19.3.0
audioread==2.1.8
autograd==1.3
Babel==2.8.0
backcall==0.1.0
beautifulsoup4==4.6.3
bleach==3.1.5
blis==0.4.1
bokeh==1.4.0
boto==2.49.0
boto3==1.13.10
botocore==1.16.10
Bottleneck==1.3.2
branca==0.4.1
bs4==0.0.1
CacheControl==0.12.6
cachetools==3.1.1
catalogue==1.0.0
certifi==2020.4.5.1
cffi==1.14.0
chainer==6.5.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.4.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.1
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.5
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.3
Cython==0.29.17
daft==0.0.4
dask==2.12.0
dataclasses==0.7
datascience==0.10.6
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.1.1
distributed==1.25.3
Django==3.0.6
dlib==19.18.0
docopt==0.6.2
docutils==0.15.2
dopami

In [0]:
# Módulos para o modelo e predição
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
from skimage.transform import resize
from keras.models import model_from_json
import cv2

# Módulos para api com flask
import pydload
import uuid
import json
import time
import requests
import os

from flask import Flask, flash, get_flashed_messages, request, redirect, url_for, render_template_string
from flask_ngrok import run_with_ngrok
#from flask_cors import CORS, cross_origin

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [0]:
# Configurando o servidor
app = Flask(__name__)
app.secret_key = b'\x19Q\xb0h0,N\xb9\x0c\xad\\\\\xde\x9d\xaf\xc5'
app.config['SESSION_TYPE'] = 'filesystem'
app.config['UPLOAD_FOLDER'] = './'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

#app.debug = True

#app.config['CORS_HEADER'] = 'Content-Type'
#cors = CORS(app, resources={r'/*': {"origins": '*'}})


In [0]:
# Funções para carregar o modelo com pesos e fazer a predição
def load_model_and_weights(path='./drive/My Drive/TCC/', model_json_name='model.json', weight_name='xray_class'):
    json_path = "{}{}".format(path, model_json_name)
    json_file = open(json_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)

    weight_path="{}{}_weights.best.hdf5".format(path, weight_name)
    # Load the model weights
    model.load_weights(weight_path)
    model._make_predict_function()
    #model.summary()
    return model

def make_prediction(img):
    img = cv2.resize(img, (224,224))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.

    pred = model.predict(np.array([img]), batch_size=32, verbose=True)
    return {
        'classification': 'Pneumonia' if pred[0][1] >= 0.85 else 'Normal',
        'prediction': 'Pneumonia: {:.09f}, Normal: {:.09f}'.format(pred[0][1], pred[0][0])
        }


In [0]:
# Caminhos da Api

#@cross_origin()
@app.route('/predict', methods=['GET', 'POST'])
def classifier_from_url():
    if request.method == 'GET':
        url = request.args.get('url')
    elif request.method == 'POST':
        url = request.json.get('url')

    try:
        path = str(uuid.uuid4())
        dload_status = pydload.dload(url, path, timeout=2 ,max_time=3)

        if not dload_status:
            return json.dumps({'error': 'File too large to download'})

        img = cv2.imread(str(path))
        os.remove(path)
        res = make_prediction(img)
        return json.dumps(res)
    except Exception as ex:
        print(ex)
        return json.dumps({'error': str(ex)})

# Função para filtrar extensão do arquivo
def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

#@cross_origin()
@app.route('/predict/upload', methods=['POST'])
def classifier_from_upload():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            return json.dumps({'error': 'No file part'})
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            return json.dumps({'error': 'No selected file'})
        if file and not allowed_file(file.filename):
            return json.dumps({'error': 'File extension not allowed'})

        try:
            # https://stackoverflow.com/questions/17170752/python-opencv-load-image-from-byte-string
            nparr = np.fromstring(file.read(), np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR) # cv2.IMREAD_COLOR in OpenCV 3.1
            res = make_prediction(img)
            return json.dumps(res)
        except Exception as ex:
            print(ex)
            return json.dumps({'error': str(ex)})

@app.route('/', methods=['GET'])
def home():
    if request.method == 'GET':
        return render_template_string(
            '''
            <!doctype html>
<title>API de Predição com Flask</title>
<h2>Selecione um arquivo para subir e classificar</h2>
<form method="post" action="{{ url_for('classifier_from_upload') }}" enctype="multipart/form-data">
    <dl>
		<p>
			<input type="file" name="file">
		</p>
    </dl>
    <p>
		  <input type="submit" value="Submit">
  	</p>
</form>
</br>

<h2>Entre com uma URL para classificar</h2>
<form method="get" action="{{ url_for('classifier_from_url') }}">
    <dl>
    <p>
      <input type="url" name="url" value="https://prod-images-static.radiopaedia.org/images/1436778/7c47ea1fb9a8510e765ef510d36012_jumbo.jpeg">
    </p>
    </dl>
    <p>
      <input type="submit" value="Submit">
    </p>
</form>
            '''
        )

if __name__=='__main__':
    # https://stackoverflow.com/questions/59741453/is-there-a-general-way-to-run-web-applications-on-google-colab
    # from google.colab.output import eval_js
    # print(eval_js("google.colab.kernel.proxyPort(6788)"))

    model = load_model_and_weights()
    run_with_ngrok(app)  # Start ngrok when app is run
    # app.run(debug=False, threaded=False, port=6788, host='0.0.0.0', use_reloader=False)
    app.run()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7a290071.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/May/2020 12:52:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2020 12:52:31] "GET /favicon.ico HTTP/1.1" 404 -
MB100% (1.0 of 1.0) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


The download will be auto-terminated in 3 if not completed.
Succefully Downloaded to: 2b5cb006-3b73-459a-ba1e-7eae293df94c
1/1 [==============================] - 0s 401ms/step


127.0.0.1 - - [14/May/2020 12:52:37] "GET /predict?url=https%3A%2F%2Fprod-images-static.radiopaedia.org%2Fimages%2F1436778%2F7c47ea1fb9a8510e765ef510d36012_jumbo.jpeg HTTP/1.1" 200 -
